<a href="https://colab.research.google.com/github/SteveCoss/SWOTdawgDISTRO/blob/main/ExplorePreliminaries_EU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook to explore SWOT preliminary steps

Confluence transforms the input datasets explored in the "Explore Datsets" notebook into formats usable for the downstream algorithms. Some of the most important steps are 1) creating timeseries files for each reach (both node and reach data are included in these files); 2) performing diagnostic checks on the input data 3)  pulling the latest gage data and adding that to the SWORD of Science.

In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
nb_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/DAWGnotebooks/Path_files', nb_path)
sys.path.insert(0,nb_path)

In [ ]:
#reset working directory to distro folder
!pwd
import os
os.chdir("/content/drive/My Drive/DAWGnotebooks/dist_5.0")
!pwd

In [ ]:
import os,sys
import datetime
import json

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd

from netCDF4 import Dataset,chartostring
from pathlib import Path

import folium

# Register pandas converters for matplotlib
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

This notebook relies on the swotdawgviz library. Use either the version included in this package or use an up-to-date version (https://github.com/klarnier/swotdawgviz)

In [ ]:
# Using embedded version of swotdawgviz
from swotdawgviz.swotdawgviz import io as sdvio
from swotdawgviz.swotdawgviz import maps as sdvm

# # Using installed version of swotdawgviz
# from swotdawgviz import io as sdvio
# from swotdawgviz import maps as sdvm

In [ ]:
# create a map so we can locate a particular reach to explore
InputDir=Path('.')

swot_dir=InputDir.joinpath('swot')
swot_nc_dir=swot_dir.joinpath('timeseries')

sword_dir=InputDir.joinpath('sword')
sword_shp_dir=sword_dir.joinpath('shp').joinpath('EU')

collection = sdvio.SwotObservationsCollection(swot_nc_dir)
print("Number of reaches: %i" % len(collection.reaches_list))


sword_hb23_reaches = sdvio.SwordShapefile(os.path.join(sword_shp_dir, "eu_sword_reaches_hb23_v11.shp"),
                                          reaches_list=collection.reaches_list)

rmap = sdvm.ReachesMap(sword_hb23_reaches.dataset)
ridmap = rmap.get_centerlines_map()
ridmap

In [ ]:
rid='24241000141' 


swotfile=swot_nc_dir.joinpath(rid + '_SWOT.nc')

swot_dataset = Dataset(swotfile)

wse=swot_dataset['reach/wse'][:]
ts = swot_dataset['reach']['time'][:]
epoch = datetime.datetime(2000,1,1,0,0,0)
tobj = [ epoch + datetime.timedelta(seconds=int(t)) for t in ts ]

fig,ax = plt.subplots()
ax.plot(tobj,wse)
plt.title('WSE timeseries for ' + rid)
plt.ylabel('Water surface elevation [m]')
plt.show()


# Prediagnostic output

Confluence runs a series of checks. 

In [ ]:
prediag_dir=InputDir.joinpath('prediagnostics').joinpath('output')
prediag_dir_list=os.listdir(prediag_dir)

print(prediag_dir_list[0:10]) 

In [ ]:
reach='23240601104'

prediag_file=prediag_dir.joinpath(reach + '_prediagnostics.nc') 

prediag_dataset = Dataset(prediag_file)

print(prediag_dataset)

In [ ]:
# let's check out the node data
prediag_dataset_node=prediag_dataset['node']

print(prediag_dataset_node)

In [ ]:
width_outliers=prediag_dataset_node['width_outliers']

fig,ax=plt.subplots()
plt.pcolor(width_outliers)
plt.xlabel('Node indices')
plt.ylabel('Time indices')
plt.colorbar()
plt.show()

In [ ]:
# check out what those outliers look like

swotfile=swot_nc_dir.joinpath(reach + '_SWOT.nc')

swot_dataset = Dataset(swotfile)

width=swot_dataset['node/width'][:]

np.shape(width)

In [ ]:
width_valid_set = width.copy()
width_valid_set[width_outliers[:,:].T == 1] = np.nan
width_outliers_set = width.copy()
width_outliers_set[width_outliers[:,:].T == 0] = np.nan

fig,ax = plt.subplots()
plt.plot(width_valid_set, c="grey", ls="-")
plt.plot(width_outliers_set[0, :], "r+", label="outliers")
plt.plot(width_outliers_set[1:, :], "r+")
plt.plot(np.median(width, axis=1), "g-", label="median")
plt.xlabel('Node indices')
plt.ylabel('Width, m')
plt.legend()
plt.show()

The missing data around nodes 10, 20 and 40 are due to an issue constructing the Ohio dataset for Verify.

# Explore gage data in the SoS

As part of the preliminary steps, Confluence pulls the latest discharge data from near-real time reporting gages. These are used to help estimate flow law parameters (constrained data product only) and to evaluate discharge accuracy.

In [ ]:
# plot up gage discharge timeseries 
sosfile_constrained=InputDir.joinpath('sos/constrained/eu_sword_v11_SOS_priors.nc')
sos_dataset_con=Dataset(sosfile_constrained)



In [ ]:
# plot up gage locations on top of the reaches
#make DF from SOS
# plot up gage locations on top of the reaches
#make DF from SOS
DEFRA_reaches=sos_dataset_con['DEFRA/DEFRA_reach_id'][:].tolist()
DEFRA_CAL=sos_dataset_con['DEFRA/CAL'][:].tolist()
DEFRA_station = chartostring(sos_dataset_con['DEFRA/DEFRA_id'][:].filled(np.nan))
DEFRA_station = [ el.strip(' ') for el in DEFRA_station ]    
DEFRAg=pd.DataFrame()
DEFRAg['Reach_ID']=DEFRA_reaches
DEFRAg['Agency_ID']=DEFRA_station 
DEFRAg['CAL']=DEFRA_CAL
DEFRAQ=np.array(sos_dataset_con['DEFRA/DEFRA_q'][:,:])
DEFRAQT=np.array(sos_dataset_con['DEFRA/DEFRA_qt'][:,:])
DEFRAg['Agency']=["DEFRA"]*len(DEFRA_CAL)

EAU_reaches=sos_dataset_con['EAU/EAU_reach_id'][:].tolist()
EAU_CAL=sos_dataset_con['EAU/CAL'][:].tolist()
EAU_station= chartostring(sos_dataset_con['EAU/EAU_id'][:].filled(np.nan))
EAU_station = [ el.strip(' ') for el in EAU_station ]   
EAUg=pd.DataFrame()
EAUg['Reach_ID']=EAU_reaches
EAUg['Agency_ID']=EAU_station
EAUg['CAL']=EAU_CAL
EAUQ=np.array(sos_dataset_con['EAU/EAU_q'][:,:])
EAUQT=np.array(sos_dataset_con['EAU/EAU_qt'][:,:])
EAUg['Agency']=["EAU"]*len(EAU_CAL)

NRTdf=pd.concat([DEFRAg,EAUg],axis=0,ignore_index=True)
SWORDncfile=InputDir.joinpath('sword/netcdf/eu_sword_v11.nc')
SWORDnc=Dataset(SWORDncfile)
SWreaches=SWORDnc['reaches/reach_id'][:]
SWreachesX=SWORDnc['reaches/x'][:]
SWreachesY=SWORDnc['reaches/y'][:]
NRTx=[]
NRTy=[]
for r in range(len(NRTdf.Reach_ID[:])):

  Ridx=np.where(NRTdf.Reach_ID[r]==np.array(SWreaches))
  NRTx.append(SWreachesX[Ridx])
  NRTy.append(SWreachesY[Ridx])

NRTdf['X']=NRTx
NRTdf['Y']=NRTy

NRTdf.crs = 'epsg:4326'
NRTdf['geometry']=gpd.points_from_xy(NRTdf.X, NRTdf.Y)

NRTdf.head()
sos_gaged_reachids=np.array(NRTdf.Reach_ID[:])
sos_gaged_agency_ids=NRTdf.Agency_ID[:]
sos_gaged_Q=[]
sos_gaged_Qt=[]
for i in range(len(DEFRAQ)+len(EAUQ)):
  if i<len(DEFRAQ):
    sos_gaged_Q.append(DEFRAQ[i])
    sos_gaged_Qt.append(DEFRAQT[i])
  else:
    sos_gaged_Q.append(EAUQ[i-len(DEFRAQ)])
    sos_gaged_Qt.append(EAUQT[i-len(DEFRAQ)])

In [ ]:
domain_reachids=collection.reaches_list
domain_reachids=[int(i) for i in domain_reachids]
# make a list of all the gaged reaches in the domain, and pull the data for the first one
gaged_domain_reaches=list()

for reachid in domain_reachids:
    if reachid in sos_gaged_reachids:
        gaged_domain_reaches.append(str(reachid))
        
        
print('Pulling data for ' + str(gaged_domain_reaches[0]) )
    
indx = np.where(sos_gaged_reachids==int(gaged_domain_reaches[0]))
Qgage=sos_gaged_Q[indx[0][0]]
tgage=sos_gaged_Qt[indx[0][0]]
gage_id=sos_gaged_agency_ids[indx[0][0]]


In [ ]:
NRTdf

In [ ]:
# limit the gage data to those that overlap with reaches in the domain
GDRID=[int(i) for i in gaged_domain_reaches]
domaindf=NRTdf.loc[NRTdf['Reach_ID'].isin(GDRID)]
domaindf.reset_index(drop=True, inplace=True)
domaindf
domaindfgeo=gpd.GeoDataFrame(domaindf)
domaindfgeo.crs = 'epsg:4326'
#domaindfgeo['geometry']=gpd.points_from_xy(domaindf.X, domaindf.Y)

In [ ]:
# map the gages

#--------------------------------
# EMBEDDED VERSION OF swotdawgviz
#--------------------------------
folium.GeoJson(data=domaindfgeo['geometry'],
                marker=folium.CircleMarker(radius = 5, # Radius in metres
                                            weight = 0, #outline weight
                                            fill_color = '#0000FF', 
                                            fill_opacity = 1),).add_to(ridmap)


#----------------------------------
# UP-TO-DATE VERSION of swotdawgviz
#----------------------------------
#gmap = sdvm.GagesMap(domaindfgeo)
#gagesmap = gmap.get_map(varname_id=None, add_to_map=ridmap)


ridmap

In [ ]:
# plot gage discharge data


indx = np.where(sos_gaged_reachids==23262000644)
Qgage=sos_gaged_Q[indx[0][0]]
tgage=sos_gaged_Qt[indx[0][0]]

# TODO: fix labels so they show dates not numbers. make sure i have the right range. does not look much like SWOT obs on gaged reach
# TODO:  possibly, plot same reach above to show temporal consistency

fig,ax= plt.subplots()
tstart=734411
tstop=734776

tplot=tgage[ ((tgage>tstart) & (tgage<tstop)).data ]
Qplot=Qgage[ ((tgage>tstart) & (tgage<tstop)).data ]

# USGS gage time in SoS is in number of days since day 1 year 1
epoch = datetime.datetime(1,1,1,0,0,0)
dtplot = [ epoch + datetime.timedelta(days=t) for t in tplot ]

ax.plot(dtplot,Qplot)
plt.title('NRT Gaged discharge for ' + str(gaged_domain_reaches[0]))
plt.ylabel('Discharge, cms')
plt.show()